In [1]:
import pandas as pd

### Preparation of Dataframes

In [2]:
org_df = pd.read_csv("organizations_processed.csv")
ppl_df = pd.read_csv("people_processed.csv")

In [3]:
acq_df = pd.read_csv("acquisitions_processed.csv")
fund_df = pd.read_csv("funds_processed.csv") #investors' investment funds
fund_rd_df = pd.read_csv("funding_rounds_processed.csv") # each funding round in the dataset
ipo_df = pd.read_csv("ipos_processed.csv") 
deg_df = pd.read_csv("degrees.csv") #people's education background
cate_gp_df = pd.read_csv("category_groups_processed.csv") #Company name with its group

df1 = pd.read_csv("event_appearances_processed_1.csv")
df2 = pd.read_csv("event_appearances_processed_2.csv")
event_app_df = df1.append(df2)
event_app_df= event_app_df.reset_index()
event_app_df= event_app_df.drop(columns=["index"]) # Event participation details

event_df= pd.read_csv("events_processed.csv") #Event details

In [4]:
df1 = pd.read_csv("jobs_processed_1.csv")
df2 = pd.read_csv("jobs_processed_2.csv")
df3 = pd.read_csv("jobs_processed_3.csv")
df4 = pd.read_csv("jobs_processed_4.csv")
df5 = pd.read_csv("jobs_processed_5.csv")
job_df = df1.append(df2)
job_df= job_df.append(df3)
job_df= job_df.append(df4)
job_df= job_df.append(df5)
job_df= job_df.reset_index()
job_df= job_df.drop(columns=["index"]) #all job and advisory roles

invest_partner_df= pd.read_csv("investment_partners_processed.csv") #Partners who are responsible for their firm's investments
investor_df= pd.read_csv("investors_processed.csv") #Active investors 
org_parent_df= pd.read_csv("org_parents_processed.csv") #Mapping between parent organizations and subsidaries

#All dataframes are now ready, org_df is the master dataframe
org_df

,uuid,country_code,region,city,status,category_list,category_groups_list,num_funding_rounds,total_funding_usd,founded_on,last_funding_on,closed_on,employee_count,primary_role,num_exits
0,cf3a40e6-920e-edfd-569e-371f84e0a4e4,USA,California,San Francisco,operating,"Finance,Financial Services,Health Care,Venture...","Financial Services,Health Care,Lending and Inv...",NaN,NaN,2015-01-01,NaN,NaN,101-250,investor,24.0
1,fa65a572-1621-dd22-57a8-92bb49217ac5,GBR,England,London,operating,"Financial Services,FinTech,Venture Capital","Financial Services,Lending and Investments",NaN,NaN,2018-01-01,NaN,NaN,11-50,investor,10.0
2,74a20af3-f4dd-6188-de60-c4ee6cd0ca4a,CHN,Zhejiang,Hangzhou,operating,"Banking,E-Commerce,Financial Services,FinTech,...","Commerce and Shopping,Financial Services,Inter...",4.0,2.200000e+10,2014-10-01,2018-06-08,NaN,5001-10000,company,12.0
3,f33a3674-ec6b-14ca-16dc-437f280dc10b,USA,Virginia,Reston,operating,"E-Commerce,Financial Services,Information Tech...","Commerce and Shopping,Financial Services,Infor...",NaN,NaN,2015-01-01,NaN,NaN,1-10,investor,19.0
4,8fa7fd0d-d5cc-425d-52cc-a2019e7d42a3,USA,California,Santa Monica,operating,"Cyber Security,Developer APIs,FinTech,Software","Financial Services,Information Technology,Priv...",1.0,NaN,2013-08-20,2014-01-23,NaN,1-10,company,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37193,ea942ecf-e2bf-43ce-9ccf-5e435b702b35,USA,Ohio,Delaware,operating,Financial Services,Financial Services,NaN,NaN,2014-01-01,NaN,NaN,11-50,company,NaN
37194,1b05416c-7c71-442b-8d1c-a63400279c68,USA,California,San Jose,operating,"Analytics,Artificial Intelligence,Financial Se...","Artificial Intelligence,Data and Analytics,Fin...",NaN,NaN,2016-01-01,NaN,NaN,11-50,company,NaN
37195,90b4a15b-69ed-4ab0-a257-e1e10b39d3c5,CAN,Ontario,Toronto,operating,"Financial Services,FinTech,Service Industry","Financial Services,Other",NaN,NaN,2018-08-12,NaN,NaN,11-50,company,NaN
37196,a6060229-6086-438e-a5e3-d6580d0105cf,USA,Maryland,Baltimore,operating,"Communities,Financial Services,Impact Investing","Community and Lifestyle,Financial Services,Len...",NaN,NaN,2020-01-01,NaN,NaN,1-10,investor,NaN


### Handling Naming Issues, and dropping some overlapped columns

In [5]:
# Some prefixes are needed due to duplicate coulmn names with other csv
# Some repeated columns existing info in other dataframes can be deleted

acq_df = acq_df.drop(columns=['acquiree_name', 'acquiree_country_code', 'acquiree_region', 'acquiree_city', 'rank'])
acq_df = acq_df.add_prefix('acquisitions_')
fund_df = fund_df.add_prefix('funds_')
ipo_df = ipo_df.add_prefix('ipo_')
fund_rd_df = fund_rd_df.drop(columns=['rank'])
fund_rd_df = fund_rd_df.add_prefix('fund_rd_')
cate_gp_df = cate_gp_df.drop(columns=['name'])
cate_gp_df = cate_gp_df.add_prefix('cat_')
job_df = job_df.add_prefix('job_')
org_parent_df = org_parent_df.add_prefix('parent_org_')

laed_investor_df = investor_df.add_prefix('lead_investor_')
partner_df = investor_df.add_prefix('partner_')
investor_df = investor_df.add_prefix('investor_')
event_df = event_df.add_prefix('event_')
event_df = event_df.rename(columns={'event_name': 'event_names'})
event_app_df = event_app_df.drop(columns=['uuid'])

deg_df = deg_df.add_prefix('degree_')
ppl_df = ppl_df.add_prefix('personal_')

### Firstly, handle investor, partners and fundings. As a partner is also an investor himself, two joinings are needed

In [6]:
# Joining Investor and Partners

#investor_df = investor_df.drop(columns=['investor_name', 'investor_country_code', 'investor_region', 'investor_city'])

invest_join_df = invest_partner_df.set_index('partner_uuid').join(partner_df.set_index('partner_uuid'))
invest_join_df = invest_join_df.set_index('investor_uuid').join(investor_df.set_index('investor_uuid'))
invest_join_df.drop(columns=['uuid'], inplace = True)

In [7]:
# Joining investor and funding rounds
fund_rd_df = fund_rd_df.set_index('fund_rd_uuid').join(invest_join_df.set_index('funding_round_uuid'))

# Joining lead investor and funding rounds
fund_rd_df = laed_investor_df.set_index('lead_investor_uuid').join(fund_rd_df.set_index('fund_rd_lead_investor_uuids'))

### Next, we handle the event. Link the event participants to the events

In [8]:
# Joining event and event_app
event_join_df = event_df.set_index('event_uuid').join(event_app_df.set_index('event_uuid'))

event_join_df.drop(columns =['permalink', 'created_at', 'updated_at','event_name'], inplace = True)

people_event_df = event_join_df.loc[event_join_df['participant_type'] == 'person']
org_event_df = event_join_df.loc[event_join_df['participant_type'] == 'organization']

### After sorting out all dataframes that are replated to "people entity", we are ready to build up a large dataframe that consists of people_uuid as index

In [9]:
#Joining People and Degrees
ppl_join = ppl_df.set_index('personal_uuid').join(deg_df.set_index('degree_person_uuid'))

In [10]:
#Joining People and People Event Participant 
ppl_join = ppl_join.join(people_event_df.set_index('participant_uuid'))

In [11]:
#Add prefix to avoid overlap of column names
ppl_join = ppl_join.add_prefix('person_')

### Now, it is time to handle organizations, put org_uuid as index

In [12]:
#Joining Organizations and IPO
org_join = org_df.set_index('uuid').join(ipo_df.set_index('ipo_org_uuid'))

In [13]:
#Joining Organizations and funds
org_join = org_join.join(fund_df.set_index('funds_entity_uuid'))

In [14]:
#Joining Organizations and fund_rounds
org_join = org_join.join(fund_rd_df.set_index('fund_rd_org_uuid'))

In [15]:
#Joining Organizations and acquisitions
org_join = org_join.join(acq_df.set_index('acquisitions_acquiree_uuid'))

In [16]:
#Joining Organizations and categories
org_join = org_join.join(cate_gp_df.set_index('cat_uuid'))

In [17]:
#Joining Organizations and its parents, if any
org_join = org_join.join(org_parent_df.set_index('parent_org_uuid'))

In [18]:
#Joining Organizations and Organization Event Participant
org_join = org_join.join(org_event_df.set_index('participant_uuid'))

### Have a look at the two large dataframes (people and organization)

In [19]:
#Drop away emtpy columns to improve efficiency
ppl_join.dropna(how = 'all', axis = 1, inplace = True)
#Drop away emtpy columns to improve efficiency
org_join.dropna(how = 'all', axis = 1, inplace = True)

In [20]:
ppl_join.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1243086 entries, 0000125e-5faf-822d-72d4-b7701e250550 to fffff797-41b1-4358-a991-7d9c51be798c
Data columns (total 28 columns):
 #   Column                                          Non-Null Count    Dtype  
---  ------                                          --------------    -----  
 0   person_personal_name                            1243086 non-null  object 
 1   person_personal_gender                          1225223 non-null  object 
 2   person_personal_country_code                    763363 non-null   object 
 3   person_personal_region                          714631 non-null   object 
 4   person_personal_city                            696569 non-null   object 
 5   person_personal_featured_job_organization_name  840183 non-null   object 
 6   person_personal_featured_job_title              840169 non-null   object 
 7   person_degree_uuid                              427634 non-null   object 
 8   person_degree_institution_uuid   

In [21]:
org_join.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44270 entries, 0001a6ec-e7e9-4d1b-8a77-adb5ac815420 to ffffabce-6d4a-b3d1-13c0-4e90cedf5270
Data columns (total 51 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   country_code                        40589 non-null  object 
 1   region                              40589 non-null  object 
 2   city                                40589 non-null  object 
 3   status                              44270 non-null  object 
 4   category_list                       44270 non-null  object 
 5   category_groups_list                44270 non-null  object 
 6   num_funding_rounds                  15804 non-null  float64
 7   total_funding_usd                   12362 non-null  float64
 8   founded_on                          44270 non-null  object 
 9   last_funding_on                     15800 non-null  object 
 10  closed_on                           656 non-n

In [22]:
#Drop away columns that are without any use to improve efficiency
org_join = org_join.drop(columns = ['ipo_uuid', 'ipo_org_name', 'funds_uuid', \
                'acquisitions_uuid', 'parent_org_parent_uuid', 'ipo_stock_symbol', 'participant_name','event_venue_name', \
                'event_short_description', 'event_description'])

### Finally, it is about the linkage between people and organization, through "job"

In [23]:
#Join Job with People
job_join = job_df.set_index('job_person_uuid').join(ppl_join)

In [24]:
#Drop away columns that are without any use to improve efficiency
job_join = job_join.drop(columns=['job_uuid', 'job_permalink', 'job_created_at', 'job_updated_at', 'person_participant_name',\
                                 'person_degree_uuid', 'person_degree_institution_uuid', 'person_event_venue_name',\
                                 'person_event_short_description', 'person_event_description'])
job_join = job_join.rename(columns={'job_person_name': 'person_name','person_event_event_roles':'person_event_roles',\
                                   'event_event_roles':'event_roles', 'event_names':'event_name', 'person_event_names': 'person_event_name'})

In [25]:
#Join Organization with Job
org_join = org_join.join(job_join.set_index('job_org_uuid'))

In [26]:
#Drop away emtpy columns to simplify the final dataframe
org_join.dropna(how = 'all', axis = 1, inplace = True)

### Have a look at the joint dataframe

In [27]:
#Let's see the info of the final DataFrame
org_join.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362955 entries, 0001a6ec-e7e9-4d1b-8a77-adb5ac815420 to ffffabce-6d4a-b3d1-13c0-4e90cedf5270
Data columns (total 70 columns):
 #   Column                                          Non-Null Count   Dtype  
---  ------                                          --------------   -----  
 0   country_code                                    357042 non-null  object 
 1   region                                          357042 non-null  object 
 2   city                                            357042 non-null  object 
 3   status                                          362955 non-null  object 
 4   category_list                                   362955 non-null  object 
 5   category_groups_list                            362955 non-null  object 
 6   num_funding_rounds                              200934 non-null  float64
 7   total_funding_usd                               181719 non-null  float64
 8   founded_on                                  

In [28]:
#The final DataFrame
org_join

,country_code,region,city,status,category_list,category_groups_list,num_funding_rounds,total_funding_usd,founded_on,last_funding_on,...,person_event_name,person_event_rank,person_event_started_on,person_event_ended_on,person_event_country_code,person_event_region,person_event_city,person_event_roles,person_participant_type,person_appearance_type
0001a6ec-e7e9-4d1b-8a77-adb5ac815420,JPN,Tokyo,Tokyo,operating,"Finance,Financial Services,Recruiting","Financial Services,Professional Services",NaN,NaN,2018-01-01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0001a6ec-e7e9-4d1b-8a77-adb5ac815420,JPN,Tokyo,Tokyo,operating,"Finance,Financial Services,Recruiting","Financial Services,Professional Services",NaN,NaN,2018-01-01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0001a6ec-e7e9-4d1b-8a77-adb5ac815420,JPN,Tokyo,Tokyo,operating,"Finance,Financial Services,Recruiting","Financial Services,Professional Services",NaN,NaN,2018-01-01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0001a6ec-e7e9-4d1b-8a77-adb5ac815420,JPN,Tokyo,Tokyo,operating,"Finance,Financial Services,Recruiting","Financial Services,Professional Services",NaN,NaN,2018-01-01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0003f244-79d0-6178-353e-33dabaf3b2c6,USA,California,San Francisco,operating,"Finance,Financial Services,Venture Capital","Financial Services,Lending and Investments",NaN,NaN,2015-06-01,NaN,...,TechCrunch Disrupt SF 2016,2255.0,12/9/2016,14/9/2016,USA,California,San Francisco,"competition,conference,expo,hackathon,meetup,n...",person,speaker
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffffabce-6d4a-b3d1-13c0-4e90cedf5270,USA,New York,New York,operating,"Banking,Enterprise Applications,Finance,Financ...","Apps,Financial Services,Lending and Investment...",6.0,1087000.0,2015-01-01,2017-11-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ffffabce-6d4a-b3d1-13c0-4e90cedf5270,USA,New York,New York,operating,"Banking,Enterprise Applications,Finance,Financ...","Apps,Financial Services,Lending and Investment...",6.0,1087000.0,2015-01-01,2017-11-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ffffabce-6d4a-b3d1-13c0-4e90cedf5270,USA,New York,New York,operating,"Banking,Enterprise Applications,Finance,Financ...","Apps,Financial Services,Lending and Investment...",6.0,1087000.0,2015-01-01,2017-11-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ffffabce-6d4a-b3d1-13c0-4e90cedf5270,USA,New York,New York,operating,"Banking,Enterprise Applications,Finance,Financ...","Apps,Financial Services,Lending and Investment...",6.0,1087000.0,2015-01-01,2017-11-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
"""
#Export out as CSV, the file is around 
org_join.to_csv('org_final_joined.csv')
"""

"\n#Export out as CSV, the file is around \norg_join.to_csv('org_final_joined.csv')\n"

In [32]:
"""
# (Primary for W2) Problem 1: Is the following items the same as organization itself? Can we come up a way to check? If yes, we can drop all duplications to simplify our df

# e.g.
#'ipo_country',vs 'fund_rd_country', vs 'country_code'
#'ipo_region', vs 'fund_rd_region', vs 'state_code'
#'ipo_city', vs 'fund_rd_city', vs 'city'
#'person_personal_rank', vs 'personal_event_rank'
#'total_funding_usd', vs 'total_funding', vs 'funds_raised_amount_usd' 
#'category_list', vs 'category_groups_list' (what should we use eaxctly......)
"""
"""
# (TBC) Problem 2: There are multiple rows for some organization eitites, due to their multiple funding rounds, or multiple people, etc.
#            I plan to keep here first, but we may have to come up with a way to handle this situation
"""
"""
# (TBC) Problem 3: The dataframe consists of a column 'Acquirer'. I chose to join using acquiriee id instead of acquirier id,
#            due to the intuition that venture companies are more likely to be acquired than to acquire others;
#            However, may need to find a way to handle the situation if there is really a startup acquiring others
"""
"""
# (TBC) Problem 4: If a startup company has a parent company, do we also need the information of its parent company in
#            our dataframes. My answer is yes and know, coz not sure how much could its parent company affect it;
#            however, after all we value a startup company all by itself. If we have to incorporate its parents' data,
#            how do we valuate its parents? We may have to build up another dataframe, or else it would be too large. 
#            Also, if its parent is an acquirer, the situation will be much more complex.
"""
"""
# (Primary for W2) Problem 5: Now, all data are raw, many NaN data and are unprocessed, we have to preprocess all the data here at once
#            Also, we have to extract those that are within 5 years and fintech field to be our target data.
"""

'\n# (Primary for W2) Problem 5: Now, all data are raw, many NaN data and are unprocessed, we have to preprocess all the data here at once\n#            Also, we have to extract those that are within 5 years and fintech field to be our target data.\n'